In [2]:
from openai import OpenAI
from pydantic import BaseModel
from googleapiclient.discovery import build
import time
import wikipedia
import wikipediaapi

In [3]:
OPENAI_API_KEY = ""
GOOGLE_API_KEY = ""
CSE_ID = ""

## Get Searching Queries

In [4]:
searching_query_system_prompt = """
You are an AI Assistant specializing in analyzing potential customer behavior through search queries. 
Your task is to analyze the provided company data and predict the search queries potential customers might use on Google to find the company's products. 
Focus on understanding the product features and customer needs, and generate queries that reflect how customers search for these products without mentioning specific company names. 
Remember, customers are searching for products, not companies.
Provide 10 queries.
"""

company_data = """
### Seiko Watch Corporation: Company Data
#### 1. Overview
- Founded: 1881 by Kintarō Hattori in Tokyo, Japan.
- Headquarters: Tokyo, Japan.
- Parent Company: Seiko Group Corporation (formerly Seiko Holdings Corporation).
- Core Business: Design, manufacture, and sale of watches, clocks, and electronic devices.
- Reputation: Known for innovation, precision, and craftsmanship. Seiko is credited with creating the world's first quartz watch (Seiko Astron, 1969) and pioneering advancements in mechanical and solar-powered watches.
- Global Presence: Operates in over 100 countries with a strong presence in Asia, North America, and Europe.
---
#### 2. Product Range
Seiko offers a diverse range of watches catering to various segments:
- Seiko 5: Affordable, durable, and stylish automatic watches for everyday wear.
- Prospex: Professional-grade sports watches for diving, aviation, and outdoor activities.
- Presage: Elegant dress watches with mechanical movements, often featuring Japanese aesthetics.
- Astron: High-end GPS solar watches with advanced technology.
- Grand Seiko: Luxury watches with exceptional craftsmanship, precision, and design (competes with Swiss luxury brands like Rolex and Omega).
- Seiko Coutura: Modern, tech-savvy watches with solar and kinetic movements.
- Seiko Essentials: Simple, affordable quartz watches for casual wear.
- Limited Editions: Collaborations and special editions (e.g., Seiko x Street Fighter, Seiko x Anime).
---
#### 3. Target Audience
- Mass Market: Budget-conscious consumers seeking reliable, affordable watches (Seiko 5, Essentials).
- Enthusiasts: Watch collectors and hobbyists interested in mechanical and automatic watches (Presage, Prospex).
- Professionals: Divers, pilots, and adventurers requiring durable, high-performance watches (Prospex).
- Luxury Consumers: Affluent buyers seeking premium craftsmanship and exclusivity (Grand Seiko).
- Tech-Savvy Individuals: Consumers interested in innovative features like solar power and GPS (Astron, Coutura).
---
#### 4. Market Position
- Global Ranking: One of the top watch brands globally, competing with Swiss brands (e.g., Rolex, Omega, TAG Heuer) and Japanese rivals (e.g., Citizen, Casio).
- Price Range: Wide spectrum, from $100 (entry-level) to $50,000+ (Grand Seiko limited editions).
- Reputation: Known for quality, innovation, and value for money. Grand Seiko is increasingly recognized as a luxury brand.
- Market Share: Strong presence in the mid-range and luxury segments, with growing popularity in emerging markets.
---
#### 5. Distribution Channels
- Retail Stores: Seiko boutiques and flagship stores (especially for Grand Seiko).
- Authorized Dealers: Partnerships with jewelry stores and watch retailers worldwide.
- E-Commerce: Official website and authorized online retailers (e.g., Amazon, Chrono24).
- Department Stores: Available in high-end department stores (e.g., Macy’s, Selfridges).
- Global Distribution Network: Strong presence in Asia, North America, Europe, and the Middle East.
---
#### 6. Marketing Strategy
- Brand Positioning: Emphasizes innovation, heritage, and Japanese craftsmanship.
- Sponsorships: Partnerships with sports events (e.g., Seiko is the official timekeeper for the Tokyo Marathon and various Olympic Games).
- Celebrity Endorsements: Collaborations with athletes and celebrities to promote specific collections.
- Digital Marketing: Active on social media platforms (Instagram, Facebook, YouTube) to engage with watch enthusiasts and showcase new releases.
- Content Marketing: Produces videos and articles highlighting the history, technology, and craftsmanship behind Seiko watches.
- Limited Editions: Creates buzz through exclusive, limited-edition models.
- Grand Seiko Strategy: Focuses on elevating Grand Seiko as a standalone luxury brand, with dedicated boutiques and marketing campaigns.
---
#### 7. Customer Feedback (General)
- Positive Feedback:
- High-quality craftsmanship and durability.
- Excellent value for money, especially in the mid-range segment.
- Reliable and accurate timekeeping (both quartz and mechanical movements).
- Appreciation for Grand Seiko’s luxury appeal and attention to detail.
- Wide variety of designs catering to different tastes and preferences.
- Negative Feedback:
- Some customers find Grand Seiko prices too high compared to Swiss luxury brands.
- Limited availability of certain models in specific regions.
- Concerns about after-sales service and warranty support in some countries.
"""

class SearchingQueries(BaseModel):
    queries: list[str]

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": searching_query_system_prompt},
        {"role": "user", "content": company_data}
    ],
    response_format=SearchingQueries
)
    
response = completion.choices[0].message
queries = response.parsed.queries

In [5]:
print(len(queries))
for query in queries:
    print(query)

10
best affordable automatic watches for everyday use
high-performance sports watches for divers
elegant dress watches with Japanese aesthetics
top GPS solar watches with advanced technology
luxury watches with exceptional craftsmanship
modern watches with solar and kinetic features
simple quartz watches for casual wear
collectible limited edition anime watches
durable watches for aviation professionals
high-end timepieces comparable to Swiss luxury brands


## Execute Google Search

In [6]:
def google_search(query, num_results=10):
    """Performs Google Search and fetches titles, links and snippets of results."""

    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)

    try:
        result = service.cse().list(q=query, cx=CSE_ID, num=num_results).execute()
        articles_info = [{
            'title': item.get('title'),
            'link': item.get('link'),
            'snippet': item.get('snippet')
        } for item in result.get('items', [])]
    except Exception as e:
        raise Exception(f"An error occurred: {e}")

    return articles_info

search_outputs = []
for query in queries:
    search_outputs.extend(google_search(query))
    time.sleep(1)


In [7]:
for search_output in search_outputs:
    print(search_output)

{'title': 'Dollar for dollar, pound for pound best mechanical watch? : r ...', 'link': 'https://www.reddit.com/r/WatchHorology/comments/1ad018b/dollar_for_dollar_pound_for_pound_best_mechanical/', 'snippet': 'Jan 28, 2024 ... And like every other watchmaking lover hates their lug length, they really have created their own bubble for an affordable in-house movement,\xa0...'}
{'title': 'What are some affordable but decent automatic watches? - Quora', 'link': 'https://www.quora.com/What-are-some-affordable-but-decent-automatic-watches', 'snippet': 'For affordable yet quality automatic watches, consider these brands: Seiko: Models like the Seiko 5 Sports offer reliability and style. Orient: Known for the\xa0...'}
{'title': '[Automatic] What are some low budget ($200-$400) entry level ...', 'link': 'https://www.reddit.com/r/Watches/comments/wp0whd/automatic_what_are_some_low_budget_200400_entry/', 'snippet': 'Aug 15, 2022 ... Seiko 5/Prospex/Presage, Citizen, and Orient all create solid wat

## Get List of Competitors (Company Names)

In [8]:
extract_competitors_system_prompt = """
You are an AI Assistant specialized in identifying potential competitors of a company. 
You will be provided with company data and search query results from customers looking for the company's products. 
Each search result includes a title, link, and snippet of the website suggested by Google.
Your task is to analyze this information and accurately identify potential competitors of the company. 
Return a list of these competitors.
"""

class Competitots(BaseModel):
    competitors: list[str]

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": extract_competitors_system_prompt},
        {"role": "user", "content": company_data},
        {"role": "user", "content": str(search_outputs)}
    ],
    response_format=SearchingQueries
)

response = completion.choices[0].message
competitors = response.parsed.queries

In [9]:
for competitor in competitors:
    print(competitor)

Orient
Citizen
Hamilton
Certina
Glycine
Steinhart
Tissot
Luminox
Obris Morgan
Garmin
Suunto
REACTOR
Bulova
Casio
Apple Watch
Rolex
Omega
TAG Heuer
Timex
G-SHOCK
Grand Seiko


## Get Competitors Data 

In [12]:
wiki = wikipediaapi.Wikipedia(
    user_agent="MySmallPythonProjectForUniversity/1.0 (martin.murzenkov@gmail.com)",
    language='en'
)

def search_wikipedia(query):
    """Searches on Wikipedia and returns the best matching page full content."""

    search_results = wikipedia.search(query)

    if not search_results:
        return ""

    best_match_title = search_results[0]
    best_match_page = wiki.page(best_match_title)
    
    if not best_match_page.exists():
        raise ValueError("The page does not exist.")

    return best_match_page.text


with open("competitors_data.txt", "w", encoding="utf-8") as file:
    for competitor in competitors:
        data = search_wikipedia(f"{competitor} watches")
        file.write(f"Competitor: {competitor}\n")
        file.write(data)
        file.write("\n\n" + "---" + "\n\n")
        time.sleep(5)

In [13]:
def count_words_in_file(file_path):
    """Counts the number of words in a given text file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        words = content.split()
        return len(words)

# Usage
file_path = "competitors_data.txt"
word_count = count_words_in_file(file_path)
print(f"The file '{file_path}' contains {word_count} words.")

The file 'competitors_data.txt' contains 55285 words.
